In [100]:
import pandas as pd
from category_encoders import CatBoostEncoder
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, ReLU, Normalization
from tensorflow.keras.optimizers import Adam  # Imp
import autokeras as ak

In [102]:
data = pd.read_csv("../R/final_model_selected.csv")

In [103]:
data.drop(['Unnamed: 0', 'Method.Ind'], axis=1, inplace=True)

In [104]:
data['Id'] = data['Id'].astype('category')

In [105]:
data = pd.concat([data.drop('Continent', axis=1),
                            pd.get_dummies(data['Continent'], prefix='Continent')], axis=1)
data = pd.concat([data.drop('Industry', axis=1),
                            pd.get_dummies(data['Industry'], prefix='Industry')], axis=1)

In [106]:
data['rolling_mean'] = data.groupby("Id")['Ghg.Change.Real'].rolling(10, min_periods=1).mean().reset_index()['Ghg.Change.Real']
data['rolling_max'] = data.groupby("Id")['Ghg.Change.Real'].rolling(10, min_periods=1).max().reset_index()['Ghg.Change.Real']
data['rolling_min'] = data.groupby("Id")['Ghg.Change.Real'].rolling(10, min_periods=1).max().reset_index()['Ghg.Change.Real']
data['rolling_count'] = data.groupby("Id")['Ghg.Change.Real'].rolling(10, min_periods=1).count().reset_index()['Ghg.Change.Real']

In [107]:
train_set = data.loc[data['Year'] < 9]
validation_set = data.loc[data['Year'] == 9]
test_set = data.loc[data['Year'] == 10]

In [108]:
X_train = train_set.drop(columns=['Ghg.Change.Real.Next'])
y_train = train_set['Ghg.Change.Real.Next'].values
X_val = validation_set.drop(columns=['Ghg.Change.Real.Next'])
y_val = validation_set['Ghg.Change.Real.Next'].values
X_test = test_set.drop(columns=['Ghg.Change.Real.Next'])
y_test = test_set['Ghg.Change.Real.Next'].values

In [109]:
cb = CatBoostEncoder()
X_train['Id'] = cb.fit_transform(X_train['Id'], y_train)
X_val['Id'] = cb.transform(X_val['Id'])
X_test['Id'] = cb.transform(X_test['Id'])

In [110]:
cb = CatBoostEncoder()
X_train['Country'] = cb.fit_transform(X_train['Country'], y_train)
X_val['Country'] = cb.transform(X_val['Country'])
X_test['Country'] = cb.transform(X_test['Country'])

In [119]:
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
X_val = X_val.astype('float32')
y_val = y_val.astype('float32')
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')

In [134]:
# Assuming X_train contains your training feature data

# Create the Normalization layer
normalizer = Normalization(axis=-1)

# Learn the statistics of the training data
normalizer.adapt(X_train)

# Define the model
model = Sequential([
    Input(shape=(162,)),
    normalizer,  # Include the Normalization layer right after the input
    Dense(32, kernel_initializer='he_normal'),  
    ReLU(),
    Dense(32, kernel_initializer='he_normal'),
    ReLU(),
    Dense(1, kernel_initializer='he_normal')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Display the model's architecture
model.summary()


2024-03-16 18:04:41.896781: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 18:04:41.910657: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normaliza  (None, 162)               325       
 tion)                                                           
                                                                 
 dense_2 (Dense)             (None, 32)                5216      
                                                                 
 re_lu_2 (ReLU)              (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 32)                1056      
                                                                 
 re_lu_3 (ReLU)              (None, 32)                0         
                                                                 
 dense_4 (Dense)             (None, 1)                 33        
                                                        

In [137]:
# Train the model
history = model.fit(
    X_train, y_train,
    epochs=100,  # Adjust the number of epochs based on your data and model complexity
    batch_size=4,  # Adjust the batch size based on your data size and computing resources
    validation_data=(X_val, y_val),  # Use X_val and y_val for validation
    verbose=1  # Set to 2 for less verbose output, or 0 for no output
)

Epoch 1/100
2751/2751 [==============================] - 15s 5ms/step - loss: 436.5552 - val_loss: 134.2869
Epoch 2/100
1113/2751 [===========>..................] - ETA: 7s - loss: 255.9082

KeyboardInterrupt: 